In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/citation_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/citation_sum


In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.9 MB 6.8 MB/s 
     |████████████████████████████████| 56 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 58.3 MB/s 
     |████████████████████████████████| 596 kB 87.8 MB/s 
     |████████████████████████████████| 895 kB 66.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=50, max_length=100, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0])


Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


My friends are cool but they eat too many carbs. I'm not a big fan of carbs, but they're good for me. I like to eat a lot of carbs. They're like my friends. I don't want to be friends with them.


In [ ]:
import pandas as pd
import numpy as np
import os

## Read the topic-focused citation contexts with their respective reference article's abstract----this is just for one field of study

In [ ]:
%mkdir BART_Results

mkdir: cannot create directory ‘BART_Results’: File exists


## Function that accepts a field of study and generates text rank based summary for each cited article

In [ ]:
fos = "machine_learning"
CITED_ID_w_TITLE_ABSTRACT_PATH = "../100_most_cited_w_titles"

df_cited_id_title = pd.read_csv(f"{CITED_ID_w_TITLE_ABSTRACT_PATH}/{fos}.csv")

In [ ]:
def _generate_BART_summary(fos, title_augment=False, topic_augment=False):
  TOPIC_FOCUSED_CITATION_CONTEXTS_PATH = f'100_highest_ranked_citing_papers/citation_contexts_TOPIC_FOCUSED/{fos}'

  for cited_id in os.listdir(TOPIC_FOCUSED_CITATION_CONTEXTS_PATH):
    cited_id_wo_csv = int(cited_id.replace('.csv', ''))
    try:
      cited_title = df_cited_id_title[df_cited_id_title['paper_id'] == cited_id_wo_csv]['paper_title'].values[0] #to be used for the title augmented summary generation
    except IndexError:
      continue

    df = pd.read_csv(f"{TOPIC_FOCUSED_CITATION_CONTEXTS_PATH}/{cited_id}")
    # Grab topic-focused citation contexts for a cited article
    if len(df) != 0:
      try:
        topic_citation_contexts_list = df['topic_focused_citation_context'].values
      except:
        continue   
      topic_citation_contexts_final = list()
      for citation_context in topic_citation_contexts_list:
        try:
          topic_citation_contexts_final.append(citation_context.replace("'", '').replace(".,", '.').replace('[', '').replace(']',''))
        except:
          continue

      # Grab cited paper's topic keywords
      cited_paper_topic_keywords = " ".join(df['cited_paper_topic_keywords'][0].split(', '))   # for augmenting topic keywords
      topic_citation_contexts_final = ' '.join(topic_citation_contexts_final)
      # get summary with BART
      if not title_augment and not topic_augment:
        #print('with no title or topic augmentation')
        #raise KeyboardInterrupt
        
        ARTICLE_TO_SUMMARIZE = topic_citation_contexts_final
        inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

        # Generate Summary
        summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=200, max_length=250, early_stopping=True)
        final_summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
        
        with open(f"BART_Results/{fos}/{cited_id}", 'w') as fp:
          fp.write(final_summary)

      elif title_augment and not topic_augment:
        #print('with title augmentation but no topic augmentation')
        #raise KeyboardInterrupt

        ARTICLE_TO_SUMMARIZE = topic_citation_contexts_final + ' ' + cited_title
        inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

        # Generate Summary
        summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=200, max_length=250, early_stopping=True)
        final_summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
        
        with open(f"BART_Results_w_title_augmentation/{fos}/{cited_id}", 'w') as fp:
          fp.write(final_summary)

      elif title_augment and topic_augment:
        print('with both title and topic augmentation')
        ARTICLE_TO_SUMMARIZE = topic_citation_contexts_final + ' ' + cited_title + ' ' + cited_paper_topic_keywords
        inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

        # Generate Summary
        summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=200, max_length=250, early_stopping=True)
        final_summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
        
        with open(f"BART_Results_w_title_and_topic_augmentation/{fos}/{cited_id}", 'w') as fp:
          fp.write(final_summary)
      

In [ ]:
fos = "machine_learning"
title_augment = True   # Turn this on/off for with title augmentation or not feature
topic_augment = True  # Turn on this one and title_augment for input configuration 3


if title_augment and not topic_augment:
  # create a folder to save the summaries generated with title augmentation
  try:
    os.mkdir(f"BART_Results_w_title_augmentation")
  except:
    pass
  try:
    os.mkdir(f"BART_Results_w_title_augmentation/{fos}")
  except:
    pass

elif not title_augment and not topic_augment:
  try:
    os.mkdir(f"BART_Results")
  except:
    pass
  try:
    os.mkdir(f"BART_Results/{fos}")
  except:
    pass

elif title_augment and topic_augment:
  # create a folder to save the summaries generated with title augmentation
  try:
    os.mkdir(f"BART_Results_w_title_and_topic_augmentation")
  except:
    pass
  try:
    os.mkdir(f"BART_Results_w_title_and_topic_augmentation/{fos}")
  except:
    pass

_generate_BART_summary(fos, title_augment, topic_augment)